# Section 7 – Game Loop & Updates (dt best practices)

In Arcade, your “game loop” lives in two core callbacks:

- `on_update(self, dt)`: **update state** (movement, timers, physics).
- `on_draw(self)`: **render** the current state (no heavy logic here).

Key ideas:
- Use **pixels per second** for speeds, and multiply by **`dt`** to get frame-rate independence.
- Keep input -> **state** (velocity, targets) -> **update with dt** -> **draw**.
- For timed events/animation, use a **time accumulator**: `acc += dt`; when `acc >= period`, trigger and subtract.

We’ll demo:
1. **Time-based motion + simple animation** using a texture flip timer.
2. **Bouncing + gravity** using velocity and acceleration with `dt`.

---

# Exercise 7.1 – Timers & dt
- Change the flip rate to 12 FPS by adjusting `frame_period`.
- Make the coin move **left** and wrap on the **left** edge.
- Pause/resume animation with a key (e.g., `P`) by skipping the animation code when paused.


In [ ]:
import arcade, math

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
TITLE = "Robot animation"

class DtBasics(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.ASH_GREY)

        self.robot_list = arcade.SpriteList()
        self.robot = arcade.Sprite(":resources:images/animated_characters/robot/robot_walk0.png", scale=1.0)
        self.robot.center_x, self.robot.center_y = 80, SCREEN_HEIGHT // 2
        self.robot_list.append(self.robot)

        self.speed_x = 180.0

        self.frames = [
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk0.png"),
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk1.png"),
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk2.png"),
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk3.png"),
        ]
        self.frame_index = 0
        self.frame_period = 0.125
        self.frame_acc = 0.0

        # wobble settings (no drift)
        self.t = 0.0
        self.wobble_amplitude_deg = 4.0 # +/- degrees
        self.wobble_speed_hz = 1.5 # cycles per second

    def on_draw(self):
        self.clear()
        self.robot_list.draw()
        arcade.draw_text(
            "Robot walks right at 180 px/s • 8 FPS walk cycle",
            16, 16, arcade.color.BLACK, 16
        )

    def on_update(self, dt):
        # Move
        self.robot.center_x += self.speed_x * dt
        if self.robot.left > SCREEN_WIDTH:
            self.robot.right = 0
        if self.robot.right < 0:
            self.robot.left = SCREEN_WIDTH

        # Animate frames
        self.frame_acc += dt
        if self.frame_acc >= self.frame_period:
            self.frame_acc -= self.frame_period
            self.frame_index = (self.frame_index + 1) % len(self.frames)
            self.robot.texture = self.frames[self.frame_index]

        # Wobble 
        self.t += dt
        self.robot.angle = self.wobble_amplitude_deg * math.sin(2 * math.pi * self.wobble_speed_hz * self.t)

window = DtBasics()
arcade.run()


# Section 7 – Game Loop & Updates (dt best practices)

In Arcade, your “game loop” lives in two core callbacks:

- `on_update(self, dt)`: **update state** (movement, timers, physics).
- `on_draw(self)`: **render** the current state (no heavy logic here).

Key ideas:
- Use **pixels per second** for speeds, and multiply by **`dt`** to get frame-rate independence.
- Keep input -> **state** (velocity, targets) -> **update with dt** -> **draw**.
- For timed events/animation, use a **time accumulator**: `acc += dt`; when `acc >= period`, trigger and subtract.

We’ll demo:
1. **Time-based motion + simple animation** using a texture flip timer.
2. **Bouncing + gravity** using velocity and acceleration with `dt`.

In [ ]:
import arcade
import random
import math

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
TITLE = "Multi-Ball Bounce — Coins + Gems, Spawns, Explosions, Trails"

BALL_COUNT = 10
GRAVITY = -900
REST = 0.95

# Explosion tuning
EXPLOSION_RADIUS = 180
EXPLOSION_STRENGTH = 2500  # try 1500–4000

TEXTURE_PATHS = [
    ":resources:images/items/coinGold.png",
    ":resources:images/items/gemBlue.png",
    ":resources:images/items/gemGreen.png",
    ":resources:images/items/gemRed.png",
]

class Ball(arcade.Sprite):
    def __init__(self, x, y):
        # pick a random coin/gem
        texture_path = random.choice(TEXTURE_PATHS)
        super().__init__(texture_path, scale=0.6)
        self.center_x = x
        self.center_y = y
        self.vx = random.uniform(-250, 250)
        self.vy = random.uniform(-100, 200)
        self.mass = 1.0

class BounceDemo(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.SKY_BLUE)

        self.ball_list = arcade.SpriteList()
        for _ in range(BALL_COUNT):
            x = random.randint(100, SCREEN_WIDTH - 100)
            y = random.randint(300, SCREEN_HEIGHT - 50)
            self.ball_list.append(Ball(x, y))

        self.ground_h = 60
        self.ground_rect = arcade.rect.LBWH(0, 0, SCREEN_WIDTH, self.ground_h)

        self.trails = []        # (x, y, alpha)
        self.explosions = []    # (x, y, radius, alpha)

    def on_draw(self):
        self.clear()
        arcade.draw_rect_filled(self.ground_rect, arcade.color.DARK_SPRING_GREEN)

        # Trails
        for x, y, alpha in self.trails:
            arcade.draw_circle_filled(x, y, 7, (255, 255, 255, int(alpha)))

        # Explosion rings
        for x, y, radius, alpha in self.explosions:
            arcade.draw_circle_outline(x, y, radius, (255, 255, 255, int(alpha)), border_width=3)

        self.ball_list.draw()

    def physics_step(self, dt):
        for ball in self.ball_list:
            ball.vy += GRAVITY * dt

            # Move
            ball.center_x += ball.vx * dt
            ball.center_y += ball.vy * dt

            # Floor bounce
            floor_y = self.ground_h + ball.height / 2
            if ball.center_y < floor_y:
                ball.center_y = floor_y
                ball.vy = -ball.vy * REST

            # Walls
            left = ball.width / 2
            right = SCREEN_WIDTH - ball.width / 2
            if ball.center_x < left or ball.center_x > right:
                ball.center_x = max(left, min(ball.center_x, right))
                ball.vx = -ball.vx * REST

            # Ceiling
            top = SCREEN_HEIGHT - ball.height / 2
            if ball.center_y > top:
                ball.center_y = top
                ball.vy = -ball.vy * REST

    def resolve_collisions(self):
        for i, a in enumerate(self.ball_list):
            for b in self.ball_list[i + 1:]:
                dx = b.center_x - a.center_x
                dy = b.center_y - a.center_y
                dist = math.hypot(dx, dy)
                # Treat as circles; 0.6 shrinks collision radius a bit
                min_dist = (a.width + b.width) / 2 * 0.6
                if 0 < dist < min_dist:
                    nx, ny = dx / dist, dy / dist
                    rvx = b.vx - a.vx
                    rvy = b.vy - a.vy
                    vel_norm = rvx * nx + rvy * ny
                    if vel_norm > 0:
                        continue
                    impulse = -(1 + REST) * vel_norm
                    impulse /= (1 / a.mass + 1 / b.mass)
                    a.vx -= (impulse / a.mass) * nx
                    a.vy -= (impulse / a.mass) * ny
                    b.vx += (impulse / b.mass) * nx
                    b.vy += (impulse / b.mass) * ny
                    overlap = min_dist - dist
                    a.center_x -= nx * overlap / 2
                    a.center_y -= ny * overlap / 2
                    b.center_x += nx * overlap / 2
                    b.center_y += ny * overlap / 2

    def on_update(self, dt):
        # Trails
        self.trails.extend((b.center_x, b.center_y, 70) for b in self.ball_list)
        self.trails = [(x, y, a - 5) for (x, y, a) in self.trails if a > 7]

        # Explosion animation (grow + fade)
        self.explosions = [
            (x, y, radius + 200 * dt, alpha - 150 * dt)
            for (x, y, radius, alpha) in self.explosions
            if alpha > 0
        ]

        self.physics_step(dt)
        self.resolve_collisions()

    def on_mouse_press(self, x, y, button, modifiers):
        if button == arcade.MOUSE_BUTTON_LEFT:
            # Spawn a random coin/gem at the cursor
            ball = Ball(x, y)
            min_y = self.ground_h + ball.height / 2 + 1
            ball.center_y = max(y, min_y)
            self.ball_list.append(ball)
        elif button == arcade.MOUSE_BUTTON_RIGHT:
            self.explode(x, y)

    def explode(self, x, y):
        # Visual ring
        self.explosions.append((x, y, 10, 255))
        # Physics push
        for b in self.ball_list:
            dx = b.center_x - x
            dy = b.center_y - y
            dist = math.hypot(dx, dy)
            if 0 < dist < EXPLOSION_RADIUS:
                nx, ny = dx / dist, dy / dist
                falloff = 1.0 - (dist / EXPLOSION_RADIUS)
                impulse = EXPLOSION_STRENGTH * falloff
                b.vx += nx * impulse
                b.vy += ny * impulse
                b.vy += 80 * falloff  # extra upward kick feels nice

window = BounceDemo()
arcade.run()